In [6]:
import os
from os_snippets import remove_empty_folders
import numpy as np
import pandas as pd
import cv2
from scipy.io import loadmat
from skimage.io import imread
from skimage.io import imsave
from skimage.exposure import rescale_intensity
from skimage.transform import resize
from skimage.util import img_as_uint
from glob import glob
from tqdm import tqdm
import matlab.engine
from all_in_focus import all_in_focus
from focal_stack_legacy import focal_stack
from dft_registration import register_with_ref
from dft_registration import register_channel
from dft_registration import interchannel_correction
from dft_registration import register_pair
from MIST_run_ijm import create_mist_command
from MIST_run_ijm import run_ijm
from stitch_meta import stitch_from_meta
from cell_segmentation import segment_cell
from cell_rna_report import generate_report
from report_xetex import integrate_report

In [7]:
BASE_DIRECTORY = r'D:\FISH_images'
BASE_DEST_DIRECTORY = r'\\10.10.10.1\NAS Processed Images'

run_id = '20211020_20210929_seq1_normal_mouseBrain_DAPI'


In [8]:
def get_tiles(d):
    """Given a directory with tile information, return a series of tile index."""
    try:
        tile_info = loadmat(os.path.join(d,'TileInfo.mat'))
    except FileNotFoundError as exc:
        raise FileNotFoundError('Tile info file not included') from exc
    tile_x = int(tile_info['TileX'])
    tile_y = int(tile_info['TileY'])
    tile_num = tile_x * tile_y
    tiles = [i+1 for i in range(tile_num)]
    return tile_x,tile_y,tiles

def try_mkdir(d):
    """Try to make a new directory d. Passes if it already exists."""
    try:
        os.makedirs(d)
    except FileExistsError:
        print(f'{d} exists.')
        pass

def patch_missing_tile(d,tiles,keyword='cyc_'):
    """Patch (possible) missing images in a given tile grid."""
    cyc_chn_list = [f for f in os.listdir(d) if f.startswith(keyword)]
    # print(glob(os.path.join(d,cyc_chn_list[0],'*.tif')))
    sample = imread(glob(os.path.join(d,cyc_chn_list[0],'*.tif'))[0])
    empty_im = np.zeros(sample.shape,dtype=np.uint16)
    for cyc_chn in cyc_chn_list:
        imgs = [f for f in os.listdir(os.path.join(d,cyc_chn)) if f.endswith('.tif')]
        for tile in tiles:
            name = f'FocalStack_{tile:03d}.tif'
            if name not in imgs:
                imsave(os.path.join(d,cyc_chn,name),empty_im,check_contrast=False)

def template_stitch(in_directory,out_directory,tile_x,tile_y,ref_chn='cy3',ref_cyc=1):
    """MIST stitching of images in template channel & cycle (unregistered)."""
    try_mkdir(out_directory)
    template_cyc_chn = f'cyc_{ref_cyc}_{ref_chn}'
    create_mist_command(tile_x,tile_y,os.path.join(in_directory,template_cyc_chn).replace('\\',r'\\\\'),out_directory.replace('\\',r'\\\\'))
    if glob(os.path.join(out_directory,f'*global*')):
        pass
    else:
        run_ijm('MIST_temp.ijm')



In [9]:
raw_directory = os.path.join(BASE_DIRECTORY,run_id)
dest_directory = os.path.join(BASE_DEST_DIRECTORY,f'{run_id}_processed')
try_mkdir(dest_directory)
aif_directory = os.path.join(dest_directory,'focal_stacked')
sdc_directory = os.path.join(dest_directory,'background_corrected')
rgs_directory = os.path.join(dest_directory,'registered')
stc_directory = os.path.join(dest_directory,'stitched')
tile_x,tile_y,tiles = get_tiles(raw_directory)
#patch_missing_tile(aif_directory,tiles,keyword='cyc_1_DAPI')
#template_stitch(rgs_directory,stc_directory,tile_x,tile_y,ref_chn='DAPI',ref_cyc=1)


\\10.10.10.1\NAS Processed Images\20211020_20210929_seq1_normal_mouseBrain_DAPI_processed exists.


In [10]:
stitch_from_meta(stc_directory,sdc_directory,tile_width=2304)

  5%|▍         | 1/22 [00:38<13:26, 38.40s/it]


FileNotFoundError: [Errno 2] No such file or directory: '\\\\10.10.10.1\\NAS Processed Images\\20211020_20210929_seq1_normal_mouseBrain_DAPI_processed\\background_corrected\\cyc_2_cy3\\FocalStack_781.tif'